In [25]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import numpy as numpy
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.models as models
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [11]:
def image_convert(tensor):
    """ Display a tensor as an image. """
    
   # image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

In [39]:
# Define the transformation to be applied to each image
# x=(x-mean)/std
#TODO why
"""Most of the pretrained models require the input to be 224x224 images. Also,
we'll need to match the normalization used when the models were trained.
Each color channel was normalized separately, the means are [0.485, 0.456, 0.406] 
and the standard deviations are [0.229, 0.224, 0.225]."""

transform = transforms.Compose([
    transforms.Resize((224,224)),
    # transforms.ToTensor(),
    # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    # transforms.Lambda(lambda x: x * 255.0)
    transforms.CenterCrop((224,224)),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

BATCH_SIZE=5
# number of subprocesses to use for data loading
num_workers = 2
image_datasets = {x: datasets.ImageFolder('./ImageByClasses', transform=transform)
                  for x in ['train', 'valid']}
                  
loaders_scratch = {
    x: torch.utils.data.DataLoader(image_datasets[x], shuffle=True, batch_size=BATCH_SIZE, num_workers=num_workers)
    for x in ['train', 'valid']}

# # Load the images from the folder
# dataset = datasets.ImageFolder('./ImageByClasses', transform=transform)

# BATCH_SIZE=5
# # Create a data loader to load the images in batches
# data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# # Iterate over the data loader to get the tensors
# for images, labels in data_loader:
#     # Do something with the tensors
#     print(images)
#     # for image in images:
#     #     # print(image_convert(image))
#     #     plt.imshow(image_convert(image))
        
    
#     # # Convert the tensor to NumPy array
#     # images_np = images.numpy()

#     # # Iterate through each image in the batch
#     for i in range(images_np.shape[0]):
#         # Extract the current image from the batch
#         current_image = images_np[i, :, :, :]

#         # Transpose the dimensions to match the order expected by matplotlib (H x W x C)
#         current_image = current_image.transpose(1, 2, 0)

#         # Display the image using matplotlib
#         plt.imshow(image_convert(images[i]))
#         plt.title(f"Image {i + 1} : {labels[i]}")
#         plt.show()
#     break


In [5]:

# class Dog(nn.Module):
#     def __init__(self,num_classes=120):
#         super(Dog, self).__init__()
#         self.vgg16 = models.vgg16(pretrained=True)  # Cargar el modelo VGG-16 preentrenado
#         # Congelar las capas para que los pesos no se actualicen durante el entrenamiento
#         for param in self.vgg16.parameters():
#             param.requires_grad = False
#         # Modificar la última capa para adaptarse al número de clases
#         in_features = self.vgg16.classifier[6].in_features
#         self.vgg16.classifier[6] = nn.Linear(in_features, num_classes)

#     def forward(self, x):
#         return self.vgg16(x)
num_classes = 120
model = models.vgg16(pretrained=True)  

# Cargar el modelo VGG-16 preentrenado
# Congelar las capas para que los pesos no se actualicen durante el entrenamiento

for param in model.features.parameters():
    param.requires_grad = False

# Modificar la última capa para adaptarse al número de clases
in_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_features, num_classes)



c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:

num_epochs = 20
batch_size = 16
learning_rate = 0.005
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(data_loader)

In [ ]:

def train(model, train_dataloader):
    #model.train() #placer le modèle en mode apprentissage
        
    for X,y in train_dataloader:#demarrage de l'apprentissage
        # X,y = X.to(device), y.to(device)
        preds = model(X)
        preds = preds[:, 0]
        erreur = objectif(preds, y )
        erreur.backward()
        sgd.step()            
        sgd.zero_grad()


In [ ]:
def predict(model, val_dataloader):
#     model.eval() #placer le modèle en mode prediction  
    all_preds, all_targets = [], []
    for X,y in val_dataloader:
     #    X,y = X.to(device), y.to(device)
        preds = model(X)
        preds = preds[:, 0]
        all_preds.append( preds )
        all_targets.append( y )
    all_preds   = torch.cat( all_preds )
    all_targets = torch.cat( all_targets )
    return all_preds, all_targets

In [26]:
cv = StratifiedKFold( n_splits=2, shuffle=True, random_state=0 )

In [27]:
def afficher(data_loader):
     for images, labels in data_loader:
          for i in range(images_np.shape[0]):
               plt.imshow(image_convert(images[i]))
               plt.title(f"Image {i + 1} : {labels[i]}")
               plt.show()
          break
     

In [42]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(data_loader)

In [40]:
def train(n_epochs, train_loader, valid_loader,
          model, optimizer, criterion, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        for batch_idx, (data, target) in enumerate(train_loader):
          #   # move to GPU
          #   if use_cuda:
          #       data, target = data.cuda(), target.cuda()
                
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            # calculate batch loss
            loss = criterion(output, target)
            # backward pass
            loss.backward()
            # parameter update
            optimizer.step()
            # update training loss
            train_loss += loss.item() * data.size(0)
            
        ######################
        # validate the model #
        ######################
        for batch_idx, (data, target) in enumerate(valid_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            ## update the average validation loss
            
            # forward pass
            output = model(data)
            # batch loss
            loss = criterion(output, target)
            # update validation loss
            valid_loss += loss.item() * data.size(0)
        
        # calculate average losses
        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(valid_loader.dataset)
        
        # print training/validation statistics 
        print('Epoch: {}\tTraining Loss: {:.6f}\t Validation Loss: {:.6f}'.
             format(epoch, train_loss, valid_loss))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).    Saving model...'.
                 format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    
    # return trained model
    return model

In [43]:
n_epochs = 20
# train the model
model = train(n_epochs, data_loaders['train'], data_loaders['valid'], model,
                     optimizer_scratch, criterion_scratch, 'model_scratch.pt')

NameError: name 'data_loaders' is not defined